In [8]:
import bs4 as bs
import requests
import os
import re
import sys
import json

In [88]:
url = 'https://pagellapolitica.it/'
uri = 'dichiarazioni/verificato?page='
pageNumber = 1
pageNumber = str(pageNumber)

In [89]:
soup = bs4.BeautifulSoup(response.text, 'lxml')
newArticles = {}
totalNumberOfArticles = 1

In [90]:
def pageLinks(pageNumber):
    pageAddress = url + uri + pageNumber
    resp = requests.get(pageAddress)
    if resp:
        source = requests.get(pageAddress).text
        soup = bs.BeautifulSoup(source, "lxml")
        tables = soup.findAll('div', {'class': 'col-lg-3 mb-7'})
        pageLinks=[]
        for table in tables:
            link = table.find('a',{'class': 'statement-link'})['href']
            articleLink = url+link
            pageLinks.append(articleLink)
    return pageLinks

In [91]:
pageLinks = pageLinks("192")
for i in pageLinks:
    print(i)

https://pagellapolitica.it/dichiarazioni/183/roberto-formigoni
https://pagellapolitica.it/dichiarazioni/187/mario-monti
https://pagellapolitica.it/dichiarazioni/188/corrado-passera
https://pagellapolitica.it/dichiarazioni/190/corrado-passera
https://pagellapolitica.it/dichiarazioni/192/beppe-grillo
https://pagellapolitica.it/dichiarazioni/206/francesco-profumo
https://pagellapolitica.it/dichiarazioni/208/roberto-maroni
https://pagellapolitica.it/dichiarazioni/209/francesco-profumo
https://pagellapolitica.it/dichiarazioni/211/renata-polverini
https://pagellapolitica.it/dichiarazioni/212/beppe-grillo
https://pagellapolitica.it/dichiarazioni/216/gianni-alemanno
https://pagellapolitica.it/dichiarazioni/215/roberto-cota


In the welcome page we see only 9 articles but when we see the HTML code, there are 16 article's link.

In [87]:
print(len(pageLinks))

12


In [6]:
import pandas as pd
parameters = ['data-prix','data-codepostal','data-idagence','data-idannonce','data-nb_chambres','data-nb_pieces','data-surface','data-typebien']
df_f = pd.DataFrame()
for par in parameters:
    l = []
    for el in em_box:
        j = el[par]
        l.append(j)
    l = pd.DataFrame(l, columns = [par])
    df_f = pd.concat([df_f,l], axis = 1)

In [7]:
print(df_f)

Empty DataFrame
Columns: [data-prix, data-codepostal, data-idagence, data-idannonce, data-nb_chambres, data-nb_pieces, data-surface, data-typebien]
Index: []
